In [0]:
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "/Formula1/includes/configuration"

Out[17]: [FileInfo(path='dbfs:/mnt/formularacedata/presentation/', name='presentation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/formularacedata/processed/', name='processed/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/formularacedata/raw/', name='raw/', size=0, modificationTime=0)]

In [0]:
%run "/Formula1/includes/common_functions"

In [0]:
schema = 'qualifyId int, raceId int, driverId int, constructorId int, number int, q1 String, q2 STring, q3 String'
file_name = f'{raw_folder_path}/qualifying/'

df = spark.read.format('json').option('multiLine',True).schema(schema).load(file_name)
df.show(3)

+---------+------+--------+-------------+------+--------+--------+--------+
|qualifyId|raceId|driverId|constructorId|number|      q1|      q2|      q3|
+---------+------+--------+-------------+------+--------+--------+--------+
|        1|    18|       1|            1|    22|1:26.572|1:25.187|1:26.714|
|        2|    18|       9|            2|     4|1:26.103|1:25.315|1:26.869|
|        3|    18|       5|            1|    23|1:25.664|1:25.452|1:27.079|
+---------+------+--------+-------------+------+--------+--------+--------+
only showing top 3 rows



In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit


df_new = df.withColumnRenamed('qualifyId','qualify_id')\
            .withColumnRenamed('raceId','race_id') \
            .withColumnRenamed('driverId','driver_id') \
            .withColumnRenamed('constructorId', 'constructor_id')  \
            .withColumn('ingestion_date',current_timestamp()) \
            .withColumn("data_source", lit(v_data_source)) \
            .withColumn("file_date", lit(v_file_date))


df_new.show(4)
            

+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+-----------+----------+
|qualify_id|race_id|driver_id|constructor_id|number|      q1|      q2|      q3|      ingestion_date|data_source| file_date|
+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+-----------+----------+
|         1|     18|        1|             1|    22|1:26.572|1:25.187|1:26.714|2023-06-29 20:50:...| Ergast_api|2021-03-21|
|         2|     18|        9|             2|     4|1:26.103|1:25.315|1:26.869|2023-06-29 20:50:...| Ergast_api|2021-03-21|
|         3|     18|        5|             1|    23|1:25.664|1:25.452|1:27.079|2023-06-29 20:50:...| Ergast_api|2021-03-21|
|         4|     18|       13|             6|     2|1:25.994|1:25.691|1:27.178|2023-06-29 20:50:...| Ergast_api|2021-03-21|
+----------+-------+---------+--------------+------+--------+--------+--------+--------------------+-----------+----------+
only sho

In [0]:
df_new.count()

Out[24]: 8694

In [0]:
merge_condition = "tgt.qualify_id = src.qualify_id AND tgt.race_id = src.race_id"
merge_delta_data(df_new, 'f1_processed', 'qualifying', processed_folder_path, merge_condition, 'race_id')


In [0]:
%sql

SELECT * FROM f1_processed.qualifying LIMIT 4

qualify_id,race_id,driver_id,constructor_id,number,q1,q2,q3,ingestion_date,data_source,file_date
4537,841,20,9,1,1:25.296,1:24.090,1:23.529,2023-06-29T20:50:41.372+0000,Ergast_api,2021-03-21
4538,841,1,1,3,1:25.384,1:24.595,1:24.307,2023-06-29T20:50:41.372+0000,Ergast_api,2021-03-21
4539,841,17,9,2,1:25.900,1:24.658,1:24.395,2023-06-29T20:50:41.372+0000,Ergast_api,2021-03-21
4540,841,18,1,4,1:25.886,1:24.957,1:24.779,2023-06-29T20:50:41.372+0000,Ergast_api,2021-03-21


In [0]:
dbutils.notebook.exit("success")

success